In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [2]:
#Extract data from pdf
def load_pdf(data):
  loader = DirectoryLoader(data,
                           glob = "*.pdf",
                           loader_cls=PyPDFLoader)
  documents = loader.load()
  return documents

In [3]:
extracted_data = load_pdf("data/")

In [4]:
#extracted_data

In [5]:
# create text chunks
def text_split(extracted_data):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20 )
  text_chunks = text_splitter.split_documents(extracted_data)
  return text_chunks
  

In [6]:
text_chunks = text_split(extracted_data)

In [7]:
print("length of my chunk : ",len(text_chunks))

length of my chunk :  5860


In [8]:
# download embedding model
def download_hugging_face_embeddings():
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
  return embedding

In [9]:
embedding = download_hugging_face_embeddings()

c:\Users\LAPSHOP\anaconda3\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\LAPSHOP\anaconda3\envs\mchatbot\lib\site-packages\transformers\modeling_utils.py:1435: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializ

In [10]:
query_result = embedding.embed_query("Hello world")
print("Length of query result : ",len(query_result))

Length of query result :  384


In [11]:
def store_data_in_faiss(text_chunks, embeddings):
    vectorstore = FAISS.from_documents(
        documents=text_chunks,
        embedding=embeddings
    )
    vectorstore.save_local("faiss_index")  # Save the index locally
    return vectorstore

In [12]:
vectorstore = store_data_in_faiss(text_chunks, embedding)

In [14]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [15]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

In [16]:
llm = CTransformers(
    model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={'max_new_tokens': 512, 'temperature': 0.8}
)

In [17]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [19]:
'''
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])
'''

'\nwhile True:\n    user_input=input(f"Input Prompt:")\n    result=qa({"query": user_input})\n    print("Response : ", result["result"])\n'